## Morpheo baseline project
# Introduction to sleep scoring
[Colaboratory version](https://colab.research.google.com/drive/1o8sjVQrexX20cv3Ca5dA5TQz_ceSS-xp)

Please execute the cell bellow to initialize the notebook environment

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import requests
import h5py
import keras

plt.rcParams.update({'figure.figsize': (4.5, 3.5)})

Using TensorFlow backend.


Please execute the cell bellow to download the sleep scoring dataset (if needed)

In [0]:
!if [ ! -d ./datasets ]; then git clone https://github.com/mpbrigham/intro-sleep-scoring; \
                            cp -rf ./intro-sleep-scoring/datasets ./; fi

## Data visualization

### Import dataset
Investigate the structure of polysomnography records in HDF5 format with `h5py` library.

**Suggestions**
* Open HDF5 database `mesa_sleep_0001_s.h5`
* Print table names and shapes

**Refs**
* h5py: HDF5 for Python (http://docs.h5py.org/en/latest)

In [0]:
path = './datasets/mesa_small/mesa_sleep_0001_s.h5'

# insert your code here

**EXPECTED OUTPUT**
```
table	 shape

EEG1  	 (1439, 1920)
EEG2  	 (1439, 1920)
EEG3  	 (1439, 1920)
EKG   	 (1439, 1920)
EMG   	 (1439, 1920)
EOG-L 	 (1439, 1920)
EOG-R 	 (1439, 1920)
stages	 (1439,)
```

### Data types
Check data type of tables and their records for EEG tables (`EEG*`) and hypnogram table (`stages`).

The hypnogram is split in 30 s intervals of recording, called *epochs*. Each epoch is assigned a sleep score.

Print data type of EEG tables and their records.

In [0]:
print('table\t table type\t\t\t\t record type\n')

# insert your code here

**EXPECTED OUTPUT**
```
table	 table type				              record type

EEG1 	 <class 'h5py._hl.dataset.Dataset'> 	 float32
EEG2 	 <class 'h5py._hl.dataset.Dataset'> 	 float32
EEG3 	 <class 'h5py._hl.dataset.Dataset'> 	 float32
stages	<class 'h5py._hl.dataset.Dataset'> 	 int32
```

### Convert data to Numpy arrays
Export EEG channels to array `x` and hypnogram to array `y`.

Print variable type of arrays `x` and `y`, and their contents.  

**Suggestions**
* Concatenate tables `EEG*` into array `x` with shape `(1439, 1920, 3)`
* Save table `stages` into array `y` with shape `(1439, 1)`

In [0]:
x, y = (None, None)

print('var\t var type\t\t\t element type\t var shape\n')

# insert your code here

**EXPECTED OUTPUT**
```
var	 var type			        element type	 var shape

x 	  <type 'numpy.ndarray'> 	 float32   	   (1439, 1920, 3)
y 	  <type 'numpy.ndarray'> 	 int32 	       (1439, 1)
```

### Visualize data
Visualize data from EEG channels and hypnogram by plotting epoch 1000 of each.

**Suggestions**
* Use functions `plt.figure()` and `plt.plot()` from Matplotlib to plot first 200 samples of epoch 1001 of array `x`. Add a small value to each channel to separate them vertically.
* Plot all samples from array `y`.

In [0]:
# insert your code here

**EXPECTED OUTPUT**

<img src="https://github.com/mpbrigham/intro-sleep-scoring/raw/master/figures/Introduction_to_sleep_scoring/eeg_time.png" style="float: left;">
<img src="https://github.com/mpbrigham/intro-sleep-scoring/raw/master/figures/Introduction_to_sleep_scoring/hypnogram_time.png" style="float: left;">

### Visualize proportion of sleep stages

Print unique elements of array y.

Print sleep stage proportions with names provided in dictionary `y_name` given below.

In [0]:
y_name = {0: 'AWA', 1:'N1', 2:'N2', 3:'N3', 4:'REM'}

# insert your code here

**EXPECTED OUTPUT**
```
unique elements of hypnogram: [0, 1, 2, 3, 4] 

AWA 	0.0000
N1 	0.0938
N2 	0.6324
N3 	0.0396
REM 	0.2168
```

### Visualize EEG samples per sleeping stage
Visualize first 200 samples of EEG from 2nd epoch of each sleeping stage.

**Suggestions**
* Use function `np.where()` from Numpy to find relevant indexes in array `y`
* Use functions `plt.subplot()` from Matplotlib to plot multiple plots in the same figure

In [0]:
fig = plt.figure(figsize=(20,3.5))

# insert your code here

**EXPECTED OUTPUT**
<img src="https://github.com/mpbrigham/intro-sleep-scoring/raw/master/figures/Introduction_to_sleep_scoring/eeg_time_stage.png" style="float: left;">

## Data pre-processing

### Basic statistical metrics
Write function `print_stats()` to print minimum, maximum, mean and standard deviation of EEG channels in array `x`.

Print unique elements of array `y` and their proportions.

**Suggestions**
* Use functions `np.min()`, `np.max()`, `np.mean()` and `np.std()` to print statistics of array `x`
* Print table of sleep stage proportions in `y`

In [0]:
def print_stats(x, name='EEG'):
    """Print minimum, maximum, mean and standard deviation along dimension 0 of array.
    x: array with shape (channel, batch, data)
    """     
    print(name+'\t min\t\t max\t\t mean\t\t std\n')

    # insert your code here

In [0]:
print_stats(x)

**EXPECTED OUTPUT**
```
EEG	 min		 max		 mean		std

0 	 -2.6387 	 2.6618 	 -0.0107 	 0.1657
1 	 -2.6356 	 2.8187 	 0.0856 	 0.1954
2 	 -6.5130 	 6.2349 	 0.0080 	 0.6050
```

### Plot histogram of EEG
Plot histogram of EEG channels in array `x`.

**Suggestions**
* Use function `np.linspace()` from Numpy to define array `x_bins` with 100 successive values between -0.2 and 0.2.
* Use function `plt.hist()` from Matplotlib to plot histogram of array `x`, using keyword `bins` to set bins location to `x_bins`

In [0]:
# insert your code here

**EXPECTED OUTPUT**

<img src="https://github.com/mpbrigham/intro-sleep-scoring/raw/master/figures/Introduction_to_sleep_scoring/eeg_histogram_pre.png" style="height: 350px;float: left;">

### Remove mean from EEG data
Write function `pre_process()` to remove channel mean from EEG channels.

Apply function to array `x`, print basic statistical metrics and plot histogram.

**Suggestions**
* Use function `np.mean()` with keyword `keepdims` to measure mean of EEG channels
* Remove mean of EEG channels from array `x`

In [0]:
def pre_process(x):
    """Remove channel mean from array x.
    x: array with shape (channel, batch, data)
    
    returns x_out: array x with zero channel mean
    """
    
    x_out = x
    
    # insert your code here
        
    return x_out

In [0]:
x = pre_process(x)

print_stats(x)

# insert your code here

**EXPECTED OUTPUT**
```
EEG	 min		 max		 mean		 std

0 	 -2.6280 	 2.6725 	 -0.0000 	 0.1657
1 	 -2.7213 	 2.7331 	 -0.0000 	 0.1954
2 	 -6.5211 	 6.2269 	 -0.0000 	 0.6050
```
<img src="https://github.com/mpbrigham/intro-sleep-scoring/raw/master/figures/Introduction_to_sleep_scoring/eeg_histogram.png" style="height: 350px;float: left;">

## Prepare data for training

### Write data import function
Write function `load_data()` to import EEG channels and hypnogram from HDF5 database.

In [0]:
def load_data(path):
    """Import EEG channels and hypnogram from HDF5 database.
    path: filesysWrite function `load_data()` to import EEG channels and hypnogram from HDF5 database.Write function `load_data()` to import EEG channels and hypnogram from HDF5 database.tem path of HDF5 database
    
    returns x: array containing EEG channels
            y: array containing hypnogram
    """
    
    x = None
    y = None
    
    # insert your code here
        
    return (x, y)

In [0]:
path = './datasets/mesa_small/mesa_sleep_0002_s.h5'

x, y = load_data(path)

x = pre_process(x)

if x is not None:
    print(x[0,:5,0])
    print(y[1000:1005])

**EXPECTED OUTPUT**
```
[0.04396349 0.07795955 0.07226041 0.07493883 0.06915694]
[[4]
 [4]
 [4]
 [4]
 [4]]
```

### Import test set
Import test set from HDF5 database `mesa_sleep_0021_s` into arrays `x_test` and `y_test`.

Print shapes of the new arrays, and basic EEG stats from array `x_test`.

In [0]:
path = './datasets/mesa_small/mesa_sleep_0021_s.h5'

x_test, y_test = load_data(path)

x_test = pre_process(x_test)

# insert your code here

**EXPECTED OUTPUT**
```
var		 shape

x_test 	 (1079, 1920, 3)
y_test 	 (1079, 1)

EEG	 min		 max		 mean		 std

0 	 -1.9979 	 0.6084 	 0.0000 	 0.0222
1 	 -0.4698 	 2.0072 	 0.0000 	 0.0380
2 	 -5.0036 	 1.0731 	 0.0000 	 0.0511
 ```

### Split dataset into train and validation sets
Split arrays `x` and `y` into train and validation sets `x_train`, `x_val`, `y_train` and `y_val`. The validation set contains 300 epochs from each HDF5 database.

Print the shapes of the new arrays.

**Note:** the function `np.random.seed(0)` from Numpy is used to replicate the expected output.

**Suggestions**
* Create boolean array `idx` with 1439 elements initialized with `False` values
* Use function `np.random.choice()` to randomly select (without replacement) 300 elements and set them to `True`
* Split `x` into `x_train` and `x_val` according to array `idx`
* Use function `np.random.seed(0)` from Numpy to replicate the expected output

In [0]:
np.random.seed(0)

x_val = None
y_val = None
x_train = None
y_train = None

# insert your code here

**EXPECTED OUTPUT**
```
var		 shape

x_train 	 (1019, 1920, 3)
y_train 	 (1019, 1)
x_val   	 (300, 1920, 3)
y_val   	 (300, 1)
```

### Generate train and validation sets
Create train and validation sets in arrays `x_train`, `y_train`, `x_val` and `y_val` from HDF5 databases `mesa_sleep_0001_s`, `mesa_sleep_0002_s`, `mesa_sleep_0006_s`, `mesa_sleep_0014_s` and `mesa_sleep_0016_s`.

Print the shapes of train and validation datasets. Print basic statistical metrics of array `x_train`.

In [0]:
np.random.seed(0)

paths = ['./datasets/mesa_small/mesa_sleep_0001_s.h5', 
         './datasets/mesa_small/mesa_sleep_0002_s.h5',
         './datasets/mesa_small/mesa_sleep_0006_s.h5', 
         './datasets/mesa_small/mesa_sleep_0014_s.h5',
         './datasets/mesa_small/mesa_sleep_0016_s.h5']

# insert your code here

**EXPECTED OUTPUT**
```
var 		shape

x_train 	 (5215, 1920, 3)
y_train 	 (5215, 1)
x_val   	 (1500, 1920, 3)
y_val   	 (1500, 1)

EEG	 min		 max		 mean		 std

0 	 -2.7610 	 2.7743 	 0.0002 	 0.1589
1 	 -2.7251 	 2.7513 	 0.0001 	 0.4217
2 	 -6.7197 	 6.6099 	 0.0006 	 0.4857
```

## Model setup

### Write input/output conversion functions
Write function `to_input()` to convert EEG data into 2-dimensional array by concatenating EEG channels on dimension 1.

i.e array `x_train` with shape `(5215, 1920, 3)` is mapped to array with shape `(5215, 5760)`.

Write function `to_output()` to sleep scores into binarized `one-hot-encoding`, using function `keras.utils.to_categorical()` from Keras library. This transformation assigns score `0` to `[1 0 0 0 0]`, score `1` to  `[0 1 0 0 0]`, etc.

i.e array `y_train` with shape `(5215, 1)` is mapped to array with shape `(5215, 5)`.

In [0]:
from keras import backend as K
from keras.layers import Input, Dense, Layer
from keras.models import Sequential

def to_input(x):
    """Convert data array to shape (batch, data).
    x: array with shape (channel, batch, data)
    
    returns x_out: array x with shape (batch, data)
    """
    
    x_out = None
    
    # insert your code here
    
    return x_out

def to_output(y):
    """Convert label array to one-hot-encoding with shape (batch, data).
    y: label array with shape (1, batch)
    
    returns: y_out (array with shape (batch, label))
    """
    
    y_out = None
    
    # insert your code here
   
    return y_out

In [0]:
if to_input(x_train) is not None:
    
    print('var\t\t\t shape\n')
    for item, item_name in ([[to_input(x_train), 'to_input(x_train)'],
                             [to_output(y_train), 'to_output(y_train)']]):
        print(item_name, '\t', item.shape)

    print('\n')
    print(to_input(x_train)[:2])
    print(to_output(y_train)[:2])

**EXPECTED OUTPUT**
```
var			 shape

to_input(x_train) 	 (5215, 5760)
to_output(y_train) 	 (5215, 5)


[[ 0.00099635  0.0048069   0.00776085 ...  0.00423072 -0.00867984
   0.02381653]
 [ 0.00212982  0.00123634 -0.00316261 ...  0.01105075  0.03953509
   0.04564268]]
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
 ```

### Convert datasets to network input/output format
Convert datasets into format compatible with network using functions `to_input()` and `to_output()`.

Print shapes of the new arrays.

In [0]:
input_train = to_input(x_train)
input_val = to_input(x_val)
input_test = to_input(x_test)

output_train = to_output(y_train)
output_val = to_output(y_val)
output_test = to_output(y_test)

if input_train is not None:
    input_shape = input_train.shape[1]
    output_shape = output_train.shape[1]

print('var\t\t shape\n')

# insert your code here

**EXPECTED OUTPUT**
```
var		 shape

input_train 	 (5215, 5760)
input_val   	 (1500, 5760)
input_test  	 (1079, 5760)
output_train	 (5215, 5)
output_val 	  (1500, 5)
output_test 	 (1079, 5)
```

### Softmax model
Implement simple network with softmax output layer with library Keras (https://keras.io/).

Write function `model_softmax()` that returns the compiled model.

Use adadelta optimizer, binary crossentropy loss and categorical accuracy metric.

In [0]:
def model_softmax():
    """Define softmax network
    
    returns m: Keras model with softmax output
    """
    
    m = None
    
    # insert your code here
    
    return m

In [0]:
model = model_softmax()

if model is not None:
    model.summary()

**EXPECTED OUTPUT**
```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input (Layer)                (None, 5760)              0         
_________________________________________________________________
output (Dense)               (None, 5)                 28805     
=================================================================
Total params: 28,805
Trainable params: 28,805
Non-trainable params: 0
_________________________________________________________________
```

### Train softmax model
Train softmax network during 5 training epochs, batch size of 32 with sample shuffling.

**Suggestions**
* Use method `fit()` with keywords `epochs`, `batch_size` and `shuffle` to train model
* Use method `evaluate()` to evaluate performance metrics in validation and test sets

In [0]:
np.random.seed(0)

n_epochs = 5

model = model_softmax()

# insert your code here

**EXPECTED OUTPUT**
```
Epoch 1/5
5215/5215 [==============================] - 2s 295us/step - loss: 0.5193 - categorical_accuracy: 0.3342
...
Epoch 5/5
5215/5215 [==============================] - 1s 162us/step - loss: 0.4387 - categorical_accuracy: 0.4086

Dataset      loss		 accuracy
val      	 0.4674 	 0.3487
test     	 0.5068 	 0.1168
```

### Performance evaluation with cross validation
Estimate model performance on unseen data with function `cross_validation()` that implements *leave-one-out cross validation*. In this  performance evaluation scheme the original dataset is split in `K` sets or *folds*. The model is succesfully trained in `K-1` sets and tested on the remaining set. In this case, a set corresponds to a database.

In [0]:
def cross_validation(paths, model_ref, epochs=5, verbose=True):
    """Leave-one-out cross validation scheme at database level
    paths: list containing paths of HDF5 databases
    model_ref: Keras model
    epochs: number of training epochs
    verbose: print intermediate results
    
    returns models: list with trained Keras models
            metrics: list with validation and test accuracy
            
    """
    
    models = []
    metrics = []
    
    # insert your code here
    
    return (models, metrics)

### Train softmax model with cross validation
Train softmax model with cross validation, 5 training epochs, batch size of 32 with sample shuffling.

In [0]:
paths = ['./datasets/mesa_small//mesa_sleep_0001_s.h5', 
         './datasets/mesa_small//mesa_sleep_0002_s.h5',
         './datasets/mesa_small//mesa_sleep_0006_s.h5', 
         './datasets/mesa_small//mesa_sleep_0014_s.h5',
         './datasets/mesa_small//mesa_sleep_0016_s.h5', 
         './datasets/mesa_small//mesa_sleep_0021_s.h5']

np.random.seed(0)
    
models, model_test = cross_validation(paths, model_softmax, epochs=n_epochs)
s
# insert your code here

**EXPECTED OUTPUT**
```
6 fold cross-validation

acc val   acc test   fold

0.3753    0.3912     1
0.3333    0.3601     2
0.3000    0.3031     3
0.4060    0.3300     4
0.3327    0.3011     5
0.3513    0.1158     6

min   	 max   	 mean   	std 	(accuracy)

0.3000 	0.4060 	0.3498 	0.0338	 val
0.1158 	0.3912 	0.3002 	0.0883	 test
```

### ANN model
Implement ANN model with single hidden layer with 256 ReLU units and softmax output.

Write function `model_ann()` that returns the compiled model.

Use adadelta optimizer, binary crossentropy loss and categorical accuracy metric.

In [0]:
def model_ann():
    """Define shallow ANN model
    
    returns m: shallow ANN Keras model
    """
    
    m = None
    
    # insert your code here
    
    return m

In [0]:
model = model_ann()

if model is not None:
    model.summary()

**EXPECTED OUTPUT**
```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input (Layer)                (None, 5760)              0         
_________________________________________________________________
h1 (Dense)                   (None, 256)               1474816   
_________________________________________________________________
output (Dense)               (None, 5)                 1285      
=================================================================
Total params: 1,476,101
Trainable params: 1,476,101
Non-trainable params: 0
_________________________________________________________________
```

### Train ANN model
Train ANN model during 5 training epochs, batch size of 32 with sample shuffling.

**Note:** the actual output may be slightly different from the expected output

In [0]:
np.random.seed(0)

n_epochs = 5

model = model_ann()

# insert your code here

**EXPECTED OUTPUT** (may vary slightly)
```
Epoch 1/5
5215/5215 [==============================] - 2s 328us/step - loss: 0.4492 - categorical_accuracy: 0.4276
...
Epoch 5/5
5215/5215 [==============================] - 1s 252us/step - loss: 0.3614 - categorical_accuracy: 0.5528

Dataset	 loss		 accuracy
val    	 0.4168 	 0.5080
test   	 0.4888 	 0.4847
```

### Train ANN model with cross validation
Train ANN model with cross validation, 5 training epochs, batch size of 32 with sample shuffling.

**Note:** the actual output may be slightly different from the expected output

In [0]:
np.random.seed(0)

models, model_test = cross_validation(paths, model_ann, epochs=n_epochs)

# insert your code here

**EXPECTED OUTPUT** (may vary slightly)
```
6 fold cross-validation

acc val   acc test   fold

0.4893    0.4246     1
0.4713    0.4049     2
0.5320    0.4551     3
0.4880    0.4586     4
0.5147    0.3661     5
0.4993    0.5088     6

min   	 max   	 mean   	std 	(accuracy)

0.4713 	0.5320 	0.4991 	0.0196	 val
0.3661 	0.5088 	0.4363 	0.0450	 test
```

## Additional questions

Make changes to the model and training data and investigate performance impacts.

**Suggestions**
* Try other types of activation units
* Add additional hidden layers 
* Use different backprop optimizer
* Change mini-batch size
* Include additional polysomnograph channels 
* Use different data pre-processing operations
* Use spectral input representation
* Use models from cross validation to implement committee of networks with majority voting